In [44]:
import numpy
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.applications import resnet_v2, vgg16, efficientnet

from model_builder import model_builder, relabel, class_merger, balancer

import model_builder
import tools_keras

from itertools import product

Load chips

In [2]:
specs = {
    'tensor': (
        '../urbangrammar_samba/'
        'spatial_signatures/chips/sample.npz'
    ),
    'folder': "../urbangrammar_samba/spatial_signatures/ai/nw_32/",
}

In [3]:
data = numpy.load(specs["tensor"], allow_pickle=True)

In [4]:
chips = data["chips"]
labels = data["labels"]

Shuffle data to ensure full randomness before splitting to test, validation and secret.

In [11]:
numpy.random.seed(0)

shuffled_idx = numpy.arange(0, chips.shape[0])
numpy.random.shuffle(shuffled_idx)

chips = chips[shuffled_idx]
labels = labels[shuffled_idx]

Merge groups

In [6]:
group_mapping = [
    ['9_0', '9_1', '9_2', '9_4', '9_5', '2_0', '2_1', '2_2'],
    ['1_0', '3_0', '5_0', '6_0', '8_0'],
    ['0_0', '4_0', '7_0']
]
labels = class_merger(labels, group_mapping)

# If you don't want to merge groups, use `relabel` to ensure your labels are in the proper format
# labels = relabel(labels)

Drop chips from excessive classes

In [7]:
mask = balancer(labels, max_ratio=10)

Total number of selected chips: 18841 out of 65042
Counts:
 {0: 907, 1: 8864, 2: 9070}


In [8]:
labels = labels[mask]
chips = chips[mask]

Rescale

In [9]:
chips = chips / (chips.max() / 255.0)

Split data into train and validation.

In [42]:
batch_size = 32
split_1 = int(chips.shape[0] * 0.6)
split_2 = int(chips.shape[0] * 0.8)


# assert that all labels are present in train and validation datasets
assert (numpy.unique(labels[:split_1]) == numpy.unique(labels[split_1:split_2])).all()
assert (numpy.unique(labels[:split_1]) == numpy.unique(labels[split_2:])).all()


train_dataset = tf.data.Dataset.from_tensor_slices((chips[:split_1], labels[:split_1]))
validation_dataset = tf.data.Dataset.from_tensor_slices((chips[split_1:split_2], labels[split_1:split_2]))
secret_dataset = tf.data.Dataset.from_tensor_slices((chips[split_2:], labels[split_2:]))


train_dataset = train_dataset.batch(batch_size=batch_size)
validation_dataset = validation_dataset.batch(batch_size=batch_size)
secret_dataset = secret_dataset.batch(batch_size=batch_size)

In [45]:
model_specs = {
    'meta_class_map': group_mapping,
    'meta_class_names': ["centres", "periphery", "countryside"],
    'meta_chip_size': 32,
}

inputs = keras.Input(shape=(32, 32, 3))
x = layers.Resizing(224, 224, crop_to_aspect_ratio=True)(inputs)
x = vgg16.preprocess_input(x)
m = keras.applications.VGG16(
    include_top=True,
    weights=None,
    input_tensor=None,
    input_shape=(224, 224, 3),
    pooling="avg",
    classes=3,
    classifier_activation="softmax")(x)
model = keras.Model(
            inputs,
            m,
            name=f"vgg16_none_none_3"
    )
# compile
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)

In [46]:
h = tools_keras.fit_phase(
        model,
        train_dataset,
        validation_dataset,
        secret_dataset,
        log_folder=specs["folder"] + "logs",
        pred_folder=specs["folder"] + "pred",
        model_folder=specs["folder"] + "model",
        json_folder=specs["folder"] + "json",
        specs=model_specs,
        epochs=250,
        patience=50,
        verbose=True,

    )

Model: "vgg16_none_none_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
resizing_3 (Resizing)        (None, 224, 224, 3)       0         
_________________________________________________________________
tf.__operators__.getitem (Sl (None, 224, 224, 3)       0         
_________________________________________________________________
tf.nn.bias_add (TFOpLambda)  (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Functional)           (None, 3)                 134272835 
Total params: 134,272,835
Trainable params: 134,272,835
Non-trainable params: 0
_________________________________________________________________
None


2022-01-17 18:22:32.331301: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-01-17 18:22:32.331369: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-01-17 18:22:32.512681: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-01-17 18:22:32.512838: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1749] CUPTI activity buffer flushed


Epoch 1/250


2022-01-17 18:22:43.466694: W tensorflow/core/common_runtime/bfc_allocator.cc:463] Allocator (GPU_0_bfc) ran out of memory trying to allocate 392.00MiB (rounded to 411041792)requested by op gradient_tape/vgg16_none_none_3/vgg16/fc1/MatMul_1
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-01-17 18:22:43.466821: I tensorflow/core/common_runtime/bfc_allocator.cc:1013] BFCAllocator dump for GPU_0_bfc
2022-01-17 18:22:43.466875: I tensorflow/core/common_runtime/bfc_allocator.cc:1020] Bin (256): 	Total Chunks: 346, Chunks in use: 345. 86.5KiB allocated for chunks. 86.2KiB in use in bin. 5.4KiB client-requested in use in bin.
2022-01-17 18:22:43.466896: I tensorflow/core/common_runtime/bfc_allocator.cc:1020] Bin (512): 	Total Chunks: 16, Chunks in use: 16. 8.2KiB allocated for chunks. 8.2KiB in use in bin. 8.0KiB client-requested in

ResourceExhaustedError:  OOM when allocating tensor with shape[25088,4096] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node gradient_tape/vgg16_none_none_3/vgg16/fc1/MatMul_1 (defined at workspace/work/signature_ai/tools_keras.py:97) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_434149]

Function call stack:
train_function
